This project was deployed using `Python 3.11` virtual environment.
- First, let's install the libraries and then
- Second, we will import the libraries that we will use in this project.

In [ ]:
%pip install -r requirements.txt

In [1]:
import os
from dotenv import load_dotenv
import sqlalchemy
import pyodbc
import pandas as pd
from pandasai import SmartDataframe
from pandasai.responses.response_parser import ResponseParser
from langchain_openai import AzureChatOpenAI
import streamlit as st

print('Project python libraries imported successfully')

Project python libraries imported successfully


Now let's import the environment variables to connect to our SQL Database.

In [2]:
load_dotenv()

## SQL Database Credentials
db_host_name=os.getenv('SQL_SERVER_NAME')
db_name=os.getenv('SQL_DB_NAME')
db_user=os.getenv('SQL_SERVER_USERNAME')
db_password=os.getenv('SQL_SERVER_PASSWORD')
odbc_driver='{ODBC Driver 18 for SQL Server}'

# Confirms that the environment variables are loaded successfully
print(f"Database Host Name: {db_host_name} imported successfully from the environment variables")


Database Host Name: barcelonaaoaipocrgr imported successfully from the environment variables


With the resources api's and credentials imported, not it is time to test the connectivity with the SQL Server:

In [4]:
# Create connection string
connection_string = {
    'drivername': 'mssql+pyodbc',
    'username': db_user,
    'password': db_password,
    'host': f"{db_host_name}.database.windows.net",
    'port': 1433,
    'database': db_name,
    'query': {'driver': 'ODBC Driver 18 for SQL Server'}
}
db_url = sqlalchemy.engine.url.URL(**connection_string)

# Connect to the database using SQLAlchemy
engine = sqlalchemy.create_engine(db_url)
connection = engine.connect()

# SQL Query to import all data from the garminactivities table, for Small table only - not for scale.
query = "SELECT * FROM [dbo].[garminactivities]"

data_df = pd.read_sql(query, connection)

connection.close()

We have a working connection to the Azure SQL Database in azure, now to keep the capacity and performance of this POC under control, let's connect to the database `aoai`, let's go to the table called `garminactivities` and let's bring all the activity records to a pandas dataframe.

In [32]:
# Creates a langchain_openai object to interact with the OpenAI API
llm = AzureChatOpenAI(
        
        model=os.getenv("OPENAI_CHAT_MODEL"),
        deployment_name=os.getenv("OPENAI_CHAT_MODEL"),
        temperature=0
    )

In [ ]:
class StreamlitResponse(ResponseParser):
    def __init__(self, context) -> None:
        super().__init__(context)

    def format_dataframe(self, result):
        st.dataframe(result["value"])
        return

    def format_plot(self, result):
        st.image(result["value"])
        return

    def format_other(self, result):
        st.write(result["value"])
        return

query_engine = SmartDataframe(
        data_df,
        config={"llm": llm,
                "response_parser": StreamlitResponse(context={})
                }
    )

summary = "Describle what is the data about. Provide a description or each column based on the header and sample from the data records."
overview = query_engine.chat(summary)
print(overview)